# Lecture 5: Mixture Models and the Expectation Maximization Algorithm

## K-means review

K-means optimizes the cost function $J$, where

$$
J(U, Z) = \sum_{n=1}^{N} \sum_{k=1}^{K} z_{k,n} \| x_n - u_k \|_2^2, \quad
\text{s.t.} \>\> z_{k,n} \in \{0, 1\} \land \sum_{k=1}{K} z_{k,n} = 1 \> \forall n
$$

Formulated as a matrix factorization, we have:

$$
\min_{U,Z} J(U, Z) = \min_{U,Z} \left\| X - UZ \right\|_F^2 
\overset{(1)}{=} \min_UZ \sum_{n=1}^N \left\| x_n - \sum_{k=1}^K z_{kn} u_k \ \right\|_2^2
$$

**(1) Tip:** We can decompose the squared Frobenius norm into squared row l2 norms, since all we're doing is adding element squares together. Note that if the norms weren't squared, we wouldn't be able to do this!

Want **probabilistic cluster assignments**. Instead of saying "this datapoint belongs to that cluster", we want to say that "this data point belongs to cluster X with probability 80%, cluster Y with probability 15%, and cluster Z with probability 5%".

**Relax z's constraint** to $z_{k,n} \in [0, 1]$, i.e. let more than one $z_{\cdot,n}$ be non-zero.

## Probabilisitic Clustering

Use **generative model == statistical model** and infer its parameters $\theta \in \Theta$ using **Maximum Likelihood**.

ML = pick model under which data has highest likelihood.


**Probabilities**: Parameter given $\implies$ data to be **generated**
 
**Likelihood**: Outcome (data) given $\implies$ parameter to be **inferred**

We turn the probability of observed data under $\theta$ *into* a likelihood function for $\theta$, given the outcome.

$$
\mathcal{L}(\theta; \mathbf{X}) := p_{\theta}(\mathbf{X}) \overset{\text{i.i.d.}}{=}
\prod_{n=1}^N p_{\theta}(x_n)
$$

We choose the **maximum likelihood estimator** $\hat{\theta}$ which maximizes the likelihood of the data, whose data points we assumed to be independently sampled from the same underlying distribution:

$$
\hat{\theta} = {\arg\max}_{\theta\in\Theta}p_{\theta}(\mathbf{X}) = 
{\arg\max}_{\theta\in\Theta} \sum_{n=1}^{N} \log p_{\theta}(\mathbf{x_n})
$$



## Mixture models

Finite mixture model for one data point:

$$
p_\theta(x) = \sum_{k=1}^K \pi_k p_{\theta_k}(x)
$$

Mixing proportions which sum up to 1. Every distribution has a proportion, **not every data point**. The whole model contains just $K$ mixing proportions. They dictate the relative cluster sizes.

GMMs are a special case where $\theta_k = \left( \mu_k, \Sigma_k \right)$.

### Gaussian Mixture Model (GMM)

$$p_{\theta_k}(x) = \mathcal{N}(\mu_k, \Sigma_k) \implies 
p_{\theta}(x) = \sum_{k=1}^K \pi_k \mathcal{N}(\mu_k, \Sigma_k)$$

For clustering, two-stage sampling. Sample a cluster, then sample from that cluster's gaussian.

􏰀Cluster index k: **latent variable**.

Final outcome x: **observed**.

We have a *graphical model* where we factor $p(x)$, the likelihood of a data point, as $p(x) = \sum_z p(x | z) p(z)$.

We introduce hard assignment variables $z_k$, BUT we denote their assignment probabilities by $\pi_k$, and work with those:

$$ P(z_k = 1) = \pi_k $$

$$ p_\pi(z) = p_\pi(z_1, \dots, z_K) = \prod_{k=1}^{K} \pi_k^{z_k} $$

Since the assignment z only has a 1 on one position by definition, the above product is actually always guaranteed to have just one term.

$$
p(x | z_k = 1) = \mathcal{N}(x | \mu_k, \Sigma_k ) \iff
p(x | z ) = \prod_{k=1}^K \mathcal{N}(x | \mu_k, \Sigma_k )^{z_k}
$$

Same as above; the second way of writing has just one term in practice.

We are now left with the following joint distribution over x and z (**complete data** distribution):

$$
p_\theta(x, z) 
\overset{\text{joint dist.}}{=} p_\theta(x | z) p_\theta(z) 
= \prod_{k=1}^K \mathcal{N}(x | \mu_k, \Sigma_k )^{z_k} \prod_{k=1}^{K} \pi_k^{z_k}
= \prod_{k=1}^K \left[ \pi_k p_{\theta_k}(x) \right]^{z_k}
$$


We can also marginalize over z:

$$
p(x) = \sum_z p(x, z) = \sum_z p(z) p(x | z) = \sum_{k=1}^K \pi_k \mathcal{N}(x|\mu_k, \Sigma_k) \overset{\text{notation}}{=} \sum_{k=1}^K \pi_k p_{\theta_k}(x)
$$

We see that for every observed data point $x_n$, there exists a latent variable (vector) $z_n$.

### Expressing the probability of an assignment

We denote $p(z_k = 1 | x) =: \gamma(z_k)$. We can compute a solution for $\gamma(z_k)$ using Bayes' theorem. $\gamma(z_k)$ is the posterior probability, after we have observed x.

$$
\gamma(z_k) = p(z_k = 1 \> | \> x) = \frac{p(x | z_k = 1) p(z_k = 1)}{\sum_{j=1}^{K} p(x | z_j = 1) p(z_j = 1)}
= \frac{\mathcal{N}(x|\mu_k, \Sigma_k) \pi_k}{\sum_{j=1}^K \mathcal{N}(x|\mu_j, \Sigma_j) \pi_j}
$$

## Maximum Likelihood

Final MLE objective: 

$$
\hat{\theta} = {\arg\max}_{\theta\in\Theta} \sum_{n=1}^{N} \log p_{\theta}(\mathbf{x_n})
 = {\arg\max}_{\theta\in\Theta} \sum_{n=1}^{N} \log \left[ \sum_{k=1}^K \pi_k p_{\theta_k}(x_n) \right]
$$

Hard to optimize directly. No closed-form solution because of the summation over k inside the logarithm, which makes the logarithm no longer act directly on the Gaussian.

### Problems with ML

When applying ML to GMMs, **singularities** can occur, and pose a problem.

If we end up, at some point, with a mean $\mu_k$ very close or equal to a data point $x_n$, then the point's contribution to the likelihood function would look like ($\Sigma_k := \sigma_k^2I$ for simplicity):

$$
\begin{aligned}
\mathcal{N}(x_n | \mu_k, \Sigma_k) & =
\left( 2 \pi \right)^{-\frac{d}{2}} |\Sigma_k|^{-\frac{1}{2}} \exp\left( -\frac{1}{2} (x_n - \mu_j)^T \Sigma_k^{-1} (x_n - \mu_j) \right) \\
 & = \left( 2 \pi \right)^{-\frac{d}{2}} (\sigma_k^2)^{-\frac{1}{2}} \exp(0) \\
 & = \frac{c}{\sigma_k} \\
\end{aligned}
$$

When $\sigma_k \to 0 $, then the point's contribution to the likelihood becomes infinite, together with the whole log likelihood. The maximization of the likelihood is therefore not a well posed problem, and special heuristics need to be used, such as resetting a component's mean and covariance whenever it becomes problematic.

## Expectation maximization

Cannot optimize log-likelihood directly (contains sum of logs). Instead we can maximize a lowe bound on the log-likelihood, based on the complete data distribution, $p_\theta(x, y)$.

### Jensen's inequality

A secant line of a convex function is always above the graph.

$$ \log\left( \frac{ \sum_{i=1}^n x_i }{n} \right)
= \log\left(\sum_{i=1}^n \frac{1}{n} x_i \right) 
\ge
= \sum_{i=1}^n \frac{1}{n} \log x_i 
= \frac{\sum_{i=1}^n\log{x_i} }{n} $$

### A. Expectation Step

TODO(andrei): See how useful the part with Jensen's inequality really is.
Optimize (maximize) **lower bound** w.r.t. "helper" distribution $q$.

Evaluate the responsabilities using the current parameter values.

$$
\gamma(z_{kn}) = \frac{\pi_k \mathcal{N}(x_n | \mu_k, \Sigma_k)}{\sum_{j=1}^K \pi_j \mathcal{N}(x_n | \mu_j, \Sigma_j) }
$$


### B. Maximization step

Can establish closed-form solutions for $\mathbf{\mu}^*$ and $\mathbf{\Sigma}^*$ given the previous $\gamma$s, as well as the data:

$$
\mu_k^{\text{new}} = \frac{1}{N_k} \sum_{n=1}^N \gamma(z_{kn}) x_n
$$

$$
\Sigma_k^{\text{new}} = \frac{1}{N_k} \sum_{n=1}^N \gamma(z_{kn}) (x_n - \mu_k^{\text{new}})(x_n - \mu_k^{\text{new}})^T
$$

The cluster weight $\pi_k$ is proportional to how much responsability a cluster assumes overall

$$
\pi_k^{\text{new}} = \frac{N_k}{N}
$$

Where:

$$
N_k = \sum_{n=1}^N \gamma(z_{nk})
$$

 * c.f. the centroid position recomputation in K-means. Remember that here, we don't have just ONE cluster assignment per point, but we have soft assignments to ALL clusters.
 * c.f. naive derivation of EM solution (see slides and Bishop)

In [1]:
# TODO(andrei): Table/diagram with precise comparison between K-means and EM.

## Model Selection

**We still need to pick K first, even in GMMs solved with the EM algorithm!!!**

One can technically keep increasing K until K==N and the LL of the data given the model keeps getting better. This is not good!

### AIC and BIC

$\kappa(\cdot)$ = number of free parameters in model.

So what does a model contain? Assuming full covariance matrices:
K \* D means, and K \* (D + 1) \* D \* 1/2 covariances PLUS K - 1 weights (-1 because we're talking about free variables, and we know they must sum to 1) $\pi$.

Note that a covariance matrix is symmetric!

#### Akaike Information Criterion

(Smaller is better)

$$ AIC(\theta | X) = -\log p_\theta(X) + \kappa(\theta) $$

#### Bayesian Information Criterion

(Smaller is better)

$$ BIC(\theta | X) = -\log p_\theta(X) + \frac{1}{2} \kappa(\theta) \log N $$

BIC is harsher.

#### Howto

Both AIC and BIC have clear minimum when computed for multiple Ks. It tends to coincide with the knee in the LL decrease.
